In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from sklearn.cluster import KMeans
import re


import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv(r'C:\Users\martin.olivares\Desktop\projects\best-route\test_3.csv')

df=pd.DataFrame()
df['address']=data['Direccion de inicio']
df['recogida']=data['Hora de recogida']

df["num_empty_cells"] = df.isna().sum(axis=1)
df["nulls"]=df['num_empty_cells']/max(df['num_empty_cells'])

df = df.drop(df[df['nulls']==1].index)
df.fillna(method='ffill', inplace=True)

df.drop(columns=['nulls','num_empty_cells'],inplace=True)


In [ ]:
# Definir un diccionario con las abreviaturas de calles y sus correspondientes formas completas
street_abbreviations = {
    "cl": "calle",
    "av": "avenida",
    "pj": "pasaje",
    "cam": "camino",
    "nte": "norte"
    }


# Definir una función que corrija las abreviaturas de calles en una dirección
def correct_address(address):
    for abbreviation, full_form in street_abbreviations.items():
        address = re.sub(r'\b{}\b'.format(abbreviation), full_form, address)
    return address

# Aplicar la función a cada dirección del DataFrame
df["address"] = df["address"].str.lower().apply(correct_address)

# Imprimir el DataFrame resultante
print(df)

                                             address recogida
0            doctor israel roizblatt 130, valparaiso     6:20
1                         ramaditas 1626, valparaíso     6:20
2                    septimo de linea 17, valparaíso     6:20
3                dionisio hernández 450,viña del mar     6:30
4                             camila 109, valparaíso     6:50
5                         ramaditas 1626, valparaíso    20:40
6            doctor israel roizblatt 130, valparaiso    20:40
7                    navío san martín 70, valparaíso    20:40
8                             camila 109, valparaíso    20:40
9   viana 1155, valparaíso, viña del mar, valparaíso    20:40
10              cancha el ocaso belloto sur, quilpue    20:30
11     irma johnson molina 2579, quilpué, valparaíso    20:30
12                      3 oriente 1270, viña del mar    20:30
13           8 norte. 1168, viña del mar, valparaíso    21:55
14           8 norte. 1168, viña del mar, valparaíso    21:55
15      

In [ ]:

#Función para verificar que las direcciones estén correctas. Posteriormente se dejan fuera las incorrectas (momentaneo)

def correct_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.address
    else:
        return 'no_existe_la_direccion'


df['corrected_address'] = df['address'].apply(correct_address)

filter=df['corrected_address']!='no_existe_la_direccion'

print(filter)

df.where(filter ,inplace=True)
df.dropna()

df


In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")

grouped = df.groupby("recogida")

for name, group in grouped:

    # Calcula la distancia entre cada par de direcciones
    X = np.zeros((len(group), len(group)))
    for i in range(len(group)):
        address1 = group.iloc[i]['address']
        loc1 = geolocator.geocode(address1)
        lat1, lon1 = loc1.latitude, loc1.longitude
        point1 = (lat1, lon1)
        for j in range(i+1, len(group)):
            address2 = group.iloc[j]['address']
            loc2 = geolocator.geocode(address2)
            lat2, lon2 = loc2.latitude, loc2.longitude
            point2 = (lat2, lon2)
            X[i, j] = great_circle(point1, point2).m
            X[j, i] = great_circle(point1, point2).m

    # Crea una matriz con las distancias
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(X)
    group['label'] = kmeans.labels_

    # Continúa agrupando hasta que cada clúster tenga 4 o menos direcciones
    while group.groupby('label').agg({'address':'count'}).max().values[0] > 4:
        kmeans.n_clusters += 1
        kmeans.fit(X)
        group['label'] = kmeans.labels_
    df.loc[group.index, "label"] = group["label"]


ValueError: n_samples=7 should be >= n_clusters=8.

In [ ]:
df

,address,recogida,corrected_address,label
0,"doctor israel roizblatt 130, valparaiso",6:20,"Doctor Israel Roizblatt, Población Dr. Israel ...",NaN
1,"ramaditas 1626, valparaíso",6:20,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",NaN
2,"septimo de linea 17, valparaíso",6:20,"Séptimo de Línea, Villa El Totoral, San Felipe...",NaN
3,"dionisio hernández 450,viña del mar",6:30,"Dionisio Hernández, Población Nuevo Horizonte,...",NaN
4,"camila 109, valparaíso",6:50,"109, Camila, Cerro San Juan de Dios, Almendral...",NaN
5,"ramaditas 1626, valparaíso",20:40,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",0.0
6,"doctor israel roizblatt 130, valparaiso",20:40,"Doctor Israel Roizblatt, Población Dr. Israel ...",0.0
7,"navío san martín 70, valparaíso",20:40,"Navío San Martín, Bosque Inglés, San Roque, Va...",0.0
8,"camila 109, valparaíso",20:40,"109, Camila, Cerro San Juan de Dios, Almendral...",0.0
9,"viana 1155, valparaíso, viña del mar, valparaíso",20:40,"Viana, Población Saenz, Forestal, Viña del Mar...",1.0
